In [1]:
import simpy

https://simpy.readthedocs.io/en/latest/contents.html

# basic

In [2]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [3]:
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


# Process interaction

In [4]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            yield self.env.process(self.charge(charge_duration))
            
            print('start driving at %d'%self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self, duration):
        yield self.env.timeout(duration)

In [5]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
start driving at 5
Start parking and charging at 7
start driving at 12
Start parking and charging at 14


In [6]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [7]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('Was interrupted. Hope, the battery is full enough ...')
            
            print('start driving at %d'%self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self, duration):
        yield self.env.timeout(duration)

In [8]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
start driving at 3
Start parking and charging at 5
start driving at 10
Start parking and charging at 12


# Resources

In [9]:
def car(env, name, bcs, driving_time, charge_duration):
    yield env.timeout(driving_time)
    
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [10]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [11]:
for i in range(4):
    env.process(car(env, 'Car %d'%i, bcs, i*2, 5))

In [12]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


# Events

In [13]:
class School:
    def __init__(self, env):
        self.env = env
        self.class_ends = env.event()
        self.pupil_procs = [env.process(self.pupil()) for i in range(3)]
        self.bell_proc = env.process(self.bell())
    
    def bell(self):
        print('start bell')
        for i in range(2):
            print('before bell timeout')
            yield self.env.timeout(45)
            print('before succeed')
            self.class_ends.succeed()
            print('after succeed')
            self.class_ends = self.env.event()
            print()
    def pupil(self):
        print('start pupil', end='')
        for i in range(2):
            print(r' \o/', end='')
            print()
            yield self.class_ends
            print(' /o\\', end='')

In [14]:
school = School(env)
env.run()

start pupil \o/
start pupil \o/
start pupil \o/
start bell
before bell timeout
before succeed
after succeed

before bell timeout
 /o\ \o/
 /o\ \o/
 /o\ \o/
before succeed
after succeed

 /o\ /o\ /o\

# Basic resource

In [15]:
def resource_user(env, resource):
    request = resource.request()
    yield request
    yield env.timeout(1)
    resource.release(request)
    
env = simpy.Environment()
res = simpy.Resource(env, capacity=1)
user = env.process(resource_user(env, res))
env.run()

In [16]:
res = simpy.Resource(env, capacity=1)
def print_stats(res):
    print('%d of %d slots are allocated.' %(res.count, res.capacity))
    print(' Users: ', res.users)
    print(' Queued events:', res.queue)
    
def user(res):
    print_stats(res)
    with res.request() as req:
        yield req
        print_stats(res)
    print_stats(res)
procs = [env.process(user(res)), env.process(user(res))]

In [17]:
env.run()

0 of 1 slots are allocated.
 Users:  []
 Queued events: []
1 of 1 slots are allocated.
 Users:  [<Request() object at 0x2a56654ec88>]
 Queued events: []
1 of 1 slots are allocated.
 Users:  [<Request() object at 0x2a56654ec88>]
 Queued events: [<Request() object at 0x2a56654ea90>]
0 of 1 slots are allocated.
 Users:  []
 Queued events: [<Request() object at 0x2a56654ea90>]
1 of 1 slots are allocated.
 Users:  [<Request() object at 0x2a56654ea90>]
 Queued events: []
0 of 1 slots are allocated.
 Users:  []
 Queued events: []


## Priority resource

In [18]:
def resource_user(name, env, resource, wait, prio):
    yield env.timeout(wait)
    with resource.request(priority=prio) as req:
        print('%s requesting at %s with priority=%s' % (name, env.now, prio))
        yield req
        print('%s got resource at %s' % (name, env.now))
        yield env.timeout(3)

In [19]:
env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
env.run()

1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
3 got resource at 3
2 got resource at 6
